# Import

In [2]:
# import required libraries for dataframe and visualization

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# import required libraries for clustering
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

# Datasets

## Reading and Understanding

In [3]:
df = pd.read_csv('kemiskinan.csv', sep=';') # If the delimiter is a semicolon

In [4]:
df.head()

,No,ID Keluarga P3KE,Dimuktakhirkan Tahun,Provinsi,Kabupaten/Kota,Kecamatan,Desa/Kelurahan,Kode Kemdagri,Desil Kesejahteraan,Persentil,...,Sumber Air Minum,Memiliki fasilitas Buang Air Besar,Resiko Stunting,Penerima BPNT,Penerima BPUM,Penerima BST,Penerima PKH,Penerima SEMBAKO,Penerima Prakerja,Penerima KUR
0,1,100469014,2021,JAWA TENGAH,KUDUS,KALIWUNGU,BAKALANKRAPYAK,3319012001,1,5,...,Sumur Bor,Milik sendiri,Bukan Target Sasaran,Ya,Tidak,Tidak,Ya,Tidak,Ya,Tidak
1,2,107530405,2021,JAWA TENGAH,KUDUS,KALIWUNGU,BAKALANKRAPYAK,3319012001,1,3,...,Sumur Terlindung,Milik sendiri,Bukan Target Sasaran,Ya,Tidak,Tidak,Tidak,Tidak,Tidak,Tidak
2,3,111876773,2021,JAWA TENGAH,KUDUS,KALIWUNGU,BAKALANKRAPYAK,3319012001,1,8,...,Sumur Terlindung,Milik sendiri,Bukan Target Sasaran,Tidak,Ya,Tidak,Tidak,Tidak,Tidak,Ya
3,4,130713310,2021,JAWA TENGAH,KUDUS,KALIWUNGU,BAKALANKRAPYAK,3319012001,1,2,...,Sumur Terlindung,Milik sendiri,Tidak Beresiko Stunting,Tidak,Ya,Tidak,Tidak,Tidak,Tidak,Tidak
4,5,189321171,2022,JAWA TENGAH,KUDUS,KALIWUNGU,BAKALANKRAPYAK,3319012001,1,7,...,Sumur Bor,Milik sendiri,Bukan Target Sasaran,Ya,Tidak,Ya,Ya,Ya,Tidak,Tidak


In [5]:
# shape of df

df.shape

(95066, 42)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95066 entries, 0 to 95065
Data columns (total 42 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   No                                               95066 non-null  int64 
 1   ID Keluarga P3KE                                 95066 non-null  int64 
 2   Dimuktakhirkan Tahun                             95066 non-null  int64 
 3   Provinsi                                         95066 non-null  object
 4   Kabupaten/Kota                                   95066 non-null  object
 5   Kecamatan                                        95066 non-null  object
 6   Desa/Kelurahan                                   95066 non-null  object
 7   Kode Kemdagri                                    95066 non-null  int64 
 8   Desil Kesejahteraan                              95066 non-null  int64 
 9   Persentil                              

In [7]:
# df description

df.describe()

,No,ID Keluarga P3KE,Dimuktakhirkan Tahun,Kode Kemdagri,Desil Kesejahteraan,Persentil
count,95066.000000,9.506600e+04,95066.000000,9.506600e+04,95066.000000,95066.000000
mean,47533.500000,1.665911e+08,2021.645783,3.319055e+09,2.620916,21.395041
std,27443.334683,3.177602e+07,0.478278,2.778325e+04,1.055652,10.825575
min,1.000000,1.000025e+08,2021.000000,3.319012e+09,1.000000,1.000000
25%,23767.250000,1.337413e+08,2021.000000,3.319032e+09,2.000000,13.000000
50%,47533.500000,1.893604e+08,2022.000000,3.319062e+09,3.000000,22.000000
75%,71299.750000,1.894273e+08,2022.000000,3.319082e+09,3.000000,30.000000
max,95066.000000,1.894914e+08,2022.000000,3.319092e+09,4.000000,40.000000


In [8]:
print(f"Is null:")
df.isna().sum().to_frame().T.style.set_properties(**{"background-color": "#FEFAE0","color":"#BC6C25","border": "1.5px  solid black"})

Is null:


,No,ID Keluarga P3KE,Dimuktakhirkan Tahun,Provinsi,Kabupaten/Kota,Kecamatan,Desa/Kelurahan,Kode Kemdagri,Desil Kesejahteraan,Persentil,Alamat,Prioritas Verval,Nama Kepala Keluarga,NIK Kepala Keluarga,NIK Kepala Keluarga Padan Kemdagri,Jenis Kelamin Kepala Keluarga,Tanggal Lahir Kepala Keluarga,Pekerjaan Kepala Keluarga,Status Pekerjaan Kepala Keluarga,Pendidikan Kepala Keluarga,Status Kawin Kepala Keluarga,Kepemilikan Rumah,Memiliki Simpanan Uang/Perhiasan/Ternak/Lainnya,Jenis Atap,Kualitas Atap,Jenis Dinding,Kualitas Dinding,Jenis Lantai,Kualitas Lantai,Sumber Penerangan,Daya Listrik Terpasang,Bahan Bakar Memasak,Sumber Air Minum,Memiliki fasilitas Buang Air Besar,Resiko Stunting,Penerima BPNT,Penerima BPUM,Penerima BST,Penerima PKH,Penerima SEMBAKO,Penerima Prakerja,Penerima KUR
0,0,0,0,0,0,0,0,0,0,0,0,0,0,405,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Data Cleansing

In [9]:
# Calculating the Missing Values % contribution in DF

df_null = round(100*(df.isnull().sum())/len(df), 2)
df_null

No                                                 0.00
ID Keluarga P3KE                                   0.00
Dimuktakhirkan Tahun                               0.00
Provinsi                                           0.00
Kabupaten/Kota                                     0.00
Kecamatan                                          0.00
Desa/Kelurahan                                     0.00
Kode Kemdagri                                      0.00
Desil Kesejahteraan                                0.00
Persentil                                          0.00
Alamat                                             0.00
Prioritas Verval                                   0.00
Nama Kepala Keluarga                               0.00
NIK Kepala Keluarga                                0.43
NIK Kepala Keluarga Padan Kemdagri                 0.00
Jenis Kelamin Kepala Keluarga                      0.00
Tanggal Lahir Kepala Keluarga                      0.00
Pekerjaan Kepala Keluarga                       

In [10]:
# Droping rows having missing values

df_new = df.dropna()
df_new.shape

(94661, 42)

In [11]:
# Daftar kolom yang ingin dibuang
columns_to_drop = ['No', 'ID Keluarga P3KE', 'Desa/Kelurahan', 'Dimuktakhirkan Tahun', 'Provinsi', 'Kabupaten/Kota', 'Kode Kemdagri', 'Alamat', 'Nama Kepala Keluarga', 'NIK Kepala Keluarga', 'Tanggal Lahir Kepala Keluarga']

# Menghapus kolom yang tidak diperlukan
df_cleaned = df_new.drop(columns=columns_to_drop)

print(df_cleaned)

       Kecamatan  Desil Kesejahteraan  Persentil  \
0      KALIWUNGU                    1          5   
1      KALIWUNGU                    1          3   
2      KALIWUNGU                    1          8   
3      KALIWUNGU                    1          2   
4      KALIWUNGU                    1          7   
...          ...                  ...        ...   
95061       DAWE                    4         36   
95062       DAWE                    4         36   
95063       DAWE                    4         33   
95064       DAWE                    4         33   
95065       DAWE                    4         39   

                     Prioritas Verval NIK Kepala Keluarga Padan Kemdagri  \
0                              Normal                        Tidak Padan   
1                              Normal                        Tidak Padan   
2                              Normal                              Padan   
3                              Normal                        Tidak Pada

In [12]:
df_cleaned.head()

,Kecamatan,Desil Kesejahteraan,Persentil,Prioritas Verval,NIK Kepala Keluarga Padan Kemdagri,Jenis Kelamin Kepala Keluarga,Pekerjaan Kepala Keluarga,Status Pekerjaan Kepala Keluarga,Pendidikan Kepala Keluarga,Status Kawin Kepala Keluarga,...,Sumber Air Minum,Memiliki fasilitas Buang Air Besar,Resiko Stunting,Penerima BPNT,Penerima BPUM,Penerima BST,Penerima PKH,Penerima SEMBAKO,Penerima Prakerja,Penerima KUR
0,KALIWUNGU,1,5,Normal,Tidak Padan,Laki-laki,Tidak/Belum Bekerja,Kosong,Tamat SD/Sederajat,Kawin,...,Sumur Bor,Milik sendiri,Bukan Target Sasaran,Ya,Tidak,Tidak,Ya,Tidak,Ya,Tidak
1,KALIWUNGU,1,3,Normal,Tidak Padan,Perempuan,Pedagang,Kosong,Tamat SD/Sederajat,Cerai Mati,...,Sumur Terlindung,Milik sendiri,Bukan Target Sasaran,Ya,Tidak,Tidak,Tidak,Tidak,Tidak,Tidak
2,KALIWUNGU,1,8,Normal,Padan,Laki-laki,Wiraswasta,Kosong,Tamat SLTA/Sederajat,Kawin,...,Sumur Terlindung,Milik sendiri,Bukan Target Sasaran,Tidak,Ya,Tidak,Tidak,Tidak,Tidak,Ya
3,KALIWUNGU,1,2,Normal,Tidak Padan,Laki-laki,Wiraswasta,Kosong,Tamat SLTP/Sederajat,Kawin,...,Sumur Terlindung,Milik sendiri,Tidak Beresiko Stunting,Tidak,Ya,Tidak,Tidak,Tidak,Tidak,Tidak
4,KALIWUNGU,1,7,Normal,Padan,Perempuan,Pensiunan,Kosong,Tamat SD/Sederajat,Cerai Hidup,...,Sumur Bor,Milik sendiri,Bukan Target Sasaran,Ya,Tidak,Ya,Ya,Ya,Tidak,Tidak


In [13]:
from sklearn.preprocessing import LabelEncoder

# List of columns to encode
columns_to_encode = ['Kecamatan','Prioritas Verval', 'NIK Kepala Keluarga Padan Kemdagri', 'Jenis Kelamin Kepala Keluarga', 'Pekerjaan Kepala Keluarga', 'Status Pekerjaan Kepala Keluarga', 'Pendidikan Kepala Keluarga', 'Status Kawin Kepala Keluarga', 'Kepemilikan Rumah', 'Memiliki Simpanan Uang/Perhiasan/Ternak/Lainnya', 'Jenis Atap', 'Kualitas Atap', 'Jenis Dinding', 'Kualitas Dinding', 'Jenis Lantai', 'Kualitas Lantai', 'Sumber Penerangan', 'Daya Listrik Terpasang', 'Bahan Bakar Memasak', 'Sumber Air Minum', 'Memiliki fasilitas Buang Air Besar', 'Resiko Stunting', 'Penerima BPNT', 'Penerima BPUM', 'Penerima BST', 'Penerima PKH', 'Penerima SEMBAKO', 'Penerima Prakerja', 'Penerima KUR']

# Apply Label Encoding to each column
label_encoders = {}
for column in columns_to_encode:
    le = LabelEncoder()
    df_cleaned[column] = le.fit_transform(df_cleaned[column])

    # Simpan label encoder untuk setiap kolom
    label_encoders[column] = le

# Tampilkan label encoder untuk satu kolom, misalnya "Pendidikan Kepala Keluarga"
print(dict(zip(label_encoders['Pendidikan Kepala Keluarga'].classes_, label_encoders['Pendidikan Kepala Keluarga'].transform(label_encoders['Pendidikan Kepala Keluarga'].classes_))))

{'Masih PT/Akademi': 0, 'Masih SD/Sederajat': 1, 'Masih SLTA/Sederajat': 2, 'Masih SLTP/Sederajat': 3, 'Tamat PT/Akademi': 4, 'Tamat SD/Sederajat': 5, 'Tamat SLTA/Sederajat': 6, 'Tamat SLTP/Sederajat': 7, 'Tdk Tamat SD/Sederajat': 8, 'Tidak/Belum Sekolah': 9}


In [14]:
for column in columns_to_encode:
    print(f"Kolom: {column}")
    print(dict(zip(label_encoders[column].classes_, label_encoders[column].transform(label_encoders[column].classes_))))
    print("\n")

Kolom: Kecamatan
{'BAE': 0, 'DAWE': 1, 'GEBOG': 2, 'JATI': 3, 'JEKULO': 4, 'KALIWUNGU': 5, 'KOTA KUDUS': 6, 'MEJOBO': 7, 'UNDAAN': 8}


Kolom: Prioritas Verval
{'Keluarga Duplikat (NIK & Nama)': 0, 'Keluarga tidak ada usia 17+ tahun': 1, 'Normal': 2}


Kolom: NIK Kepala Keluarga Padan Kemdagri
{'Padan': 0, 'Tidak Padan': 1}


Kolom: Jenis Kelamin Kepala Keluarga
{'Laki-laki': 0, 'Perempuan': 1}


Kolom: Pekerjaan Kepala Keluarga
{'Nelayan': 0, 'PNS/TNI/Polri': 1, 'Pedagang': 2, 'Pegawai Swasta': 3, 'Pejabat Negara': 4, 'Pekerja Lepas': 5, 'Pensiunan': 6, 'Petani': 7, 'Tidak/Belum Bekerja': 8, 'Wiraswasta': 9}


Kolom: Status Pekerjaan Kepala Keluarga
{'Berusaha Sendiri': 0, 'Berusaha dibantu Buruh Tetap/Dibayar': 1, 'Berusaha dibantu Buruh Tidak Tetap/Buruh Tidak Dibayar': 2, 'Buruh/Karyawan/Pegawai': 3, 'Kosong': 4, 'Pekerja Bebas': 5, 'Pekerja Keluarga/Tidak Dibayar': 6}


Kolom: Pendidikan Kepala Keluarga
{'Masih PT/Akademi': 0, 'Masih SD/Sederajat': 1, 'Masih SLTA/Sederajat': 2, 'M

In [15]:
df_cleaned.head()

,Kecamatan,Desil Kesejahteraan,Persentil,Prioritas Verval,NIK Kepala Keluarga Padan Kemdagri,Jenis Kelamin Kepala Keluarga,Pekerjaan Kepala Keluarga,Status Pekerjaan Kepala Keluarga,Pendidikan Kepala Keluarga,Status Kawin Kepala Keluarga,...,Sumber Air Minum,Memiliki fasilitas Buang Air Besar,Resiko Stunting,Penerima BPNT,Penerima BPUM,Penerima BST,Penerima PKH,Penerima SEMBAKO,Penerima Prakerja,Penerima KUR
0,5,1,5,2,1,0,8,4,5,3,...,5,2,1,1,0,0,1,0,1,0
1,5,1,3,2,1,1,2,4,5,2,...,6,2,1,1,0,0,0,0,0,0
2,5,1,8,2,0,0,9,4,6,3,...,6,2,1,0,1,0,0,0,0,1
3,5,1,2,2,1,0,9,4,7,3,...,6,2,2,0,1,0,0,0,0,0
4,5,1,7,2,0,1,6,4,5,1,...,5,2,1,1,0,1,1,1,0,0


In [16]:
# Mengubah seluruh kolom menjadi int64
df_cleaned = df_cleaned.astype('int64')

In [17]:
print(f"Shape: {df_cleaned.shape}")
print(f"\nDataset in a nutshell:")
df_cleaned.head().style.set_properties(**{"background-color": "#FEFAE0","color": "#BC6C25","border": "1.5px  solid black"})

Shape: (94661, 31)

Dataset in a nutshell:


,Kecamatan,Desil Kesejahteraan,Persentil,Prioritas Verval,NIK Kepala Keluarga Padan Kemdagri,Jenis Kelamin Kepala Keluarga,Pekerjaan Kepala Keluarga,Status Pekerjaan Kepala Keluarga,Pendidikan Kepala Keluarga,Status Kawin Kepala Keluarga,Kepemilikan Rumah,Memiliki Simpanan Uang/Perhiasan/Ternak/Lainnya,Jenis Atap,Kualitas Atap,Jenis Dinding,Kualitas Dinding,Jenis Lantai,Kualitas Lantai,Sumber Penerangan,Daya Listrik Terpasang,Bahan Bakar Memasak,Sumber Air Minum,Memiliki fasilitas Buang Air Besar,Resiko Stunting,Penerima BPNT,Penerima BPUM,Penerima BST,Penerima PKH,Penerima SEMBAKO,Penerima Prakerja,Penerima KUR
0,5,1,5,2,1,0,8,4,5,3,4,2,3,2,4,2,2,2,1,0,2,5,2,1,1,0,0,1,0,1,0
1,5,1,3,2,1,1,2,4,5,2,4,1,3,2,4,2,2,2,1,1,2,6,2,1,1,0,0,0,0,0,0
2,5,1,8,2,0,0,9,4,6,3,4,2,3,2,4,2,2,2,1,0,2,6,2,1,0,1,0,0,0,0,1
3,5,1,2,2,1,0,9,4,7,3,4,1,3,2,4,2,2,2,1,0,2,6,2,2,0,1,0,0,0,0,0
4,5,1,7,2,0,1,6,4,5,1,4,2,3,0,4,0,2,0,1,0,2,5,2,1,1,0,1,1,1,0,0


In [18]:
kmeans = KMeans(n_clusters=4, random_state=42)